In [ ]:
#pip install scipy
#pip install shapely
#pip install dash
#pip install networkx
#pip install scikit-learn

# 🚀 Experiment Execution Flow

In [ ]:
import sys
from pathlib import Path

base_dir = Path.cwd().parent
src_path = base_dir / "src"

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

    
from project_setup import (
    setup_paths,
    load_experiments_dict,
    import_and_reload_modules
)

# Set paths and load experiments
base_dir, src_path = setup_paths()
experiments_definition = load_experiments_dict("experiments_config.json")

# Import project classes
WorldModel, DisplayManager, MatrixSimilarity, NetworkManager = import_and_reload_modules()

# Plotly settings
import plotly.io as pio
pio.renderers.default = "iframe_connected"

# Experiment configuration
experiment_id = "WS16"
experiment_display_scale = {
    "ECIR19": 0.28,
    "ECSS18": 0.15,
    "ICCSS17": 0.3,
    "WS16": 0.15
}

# Initialize world & displayer
world = WorldModel(experiment_id, experiments_definition, base_dir)
world.initialize()

Displayer = DisplayManager(world)


## 🔄 Period Detection & Adjustment Pipeline

In [ ]:
# 🔍 Detect Transitions (Activity-based)
debug_data = world.period_manager.detect_transitions(
    threshold_pos=0.1, threshold_neg=-0.05, smooth_sigma=2, freq="1min", debug=False
)
if debug_data:
    Displayer.plotter.plot_transition_debug_activity(
        activity_series=debug_data["activity_series"],
        smoothed=debug_data["smoothed"],
        norm_derivative=debug_data["norm_derivative"],
        transition_df=debug_data["transition_df"],
        experiment_id=world.experiment_id,
        smooth_sigma=debug_data["smooth_sigma"],
    )

In [ ]:
simil = MatrixSimilarity()
activity_signature = simil.compute_activity_matrix(
    world.period_manager.final_period_df, world.signatures
)

cosine_signature, signature_group = simil.compute_cosine_similarity(
    activity_input=activity_signature,
    reordered=True,
    num_groups=8,
    return_groups=True,
)
world.define_areas_by_group(signature_group)


In [ ]:
world.assign_agents_to_areas_over_time(freq="20s")

In [ ]:
world.compute_active_agent_to_area()

In [ ]:
Networker = NetworkManager()
Networker.build_area_transition_graph(world)
Networker.build_temporal_area_transition_graphs(world)

In [ ]:
Networker.build_temporal_agent_graphs_by_area(world,'20s')

In [ ]:
Displayer.mapper.display(scale=experiment_display_scale[experiment_id])

In [ ]:
Displayer.networker.plot_node_edge_distribution_per_area(world)

In [ ]:
Displayer.networker.plot_area_transition_sankey(Networker.area_transition_graph,world)

In [ ]:
world.compute_agents_entropies()

In [ ]:
Displayer.plotter.plot_agent_entropy_distribution(world.agents)

In [ ]:
# === 🔄 Networker Method ===
Displayer.networker.run_dash(Networker.area_transition_graph)

# === 📈 Plotter Methods ===
Displayer.plotter.plot_normalized_activity(
    entities=world.areas,
    experiment_id=world.experiment_id,
    mode_label="areas",
    title_suffix="",
    show_total=True,
    sort_by_activity=True,
    df_period=world.period_manager.final_period_df,
)

Displayer.plotter.plot_signature_distributions()

Displayer.plotter.periods_df = world.period_manager.final_period_df


Displayer.plotter.report_world_model()
Displayer.plotter.report_periods()

# === 🧩 Matrixer Method ===
Displayer.matrixer.plot_cosine_similarity_matrix(
    cos_sim_df=cosine_signature,
    title=f"Cosine Similarity Between Signatures — Experiment: {experiment_id}",
    cmap="Viridis",
    linewidth=0.5,
    linecolor="gray",
    annot=False,
    fmt=".2f",
)

# === 🗺️ Mapper Methods ===
Displayer.mapper.display(scale=experiment_display_scale[experiment_id])

Displayer.mapper.plot_animated_signature_activity(
    grid_size=(20, 20),
    sigma=0.4,
    scale=experiment_display_scale[experiment_id],
    heatmap_opacity=0.8,
    time_bin="30min"
)
